In [3]:
import pandas as pd

df = pd.read_excel (r'Financial Sample.xlsx')
print (df)

              Segment                   Country    Product Discount Band  \
0          Government                    Canada  Carretera          None   
1          Government                   Germany  Carretera          None   
2           Midmarket                    France  Carretera          None   
3           Midmarket                   Germany  Carretera          None   
4           Midmarket                    Mexico  Carretera          None   
..                ...                       ...        ...           ...   
695    Small Business                    France   Amarilla          High   
696    Small Business                    Mexico   Amarilla          High   
697        Government                    Mexico    Montana          High   
698        Government                    Canada      Paseo          High   
699  Channel Partners  United States of America        VTT          High   

     Units Sold  Manufacturing Price  Sale Price  Gross Sales  Discounts  \
0        16

In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[107]:


import os
import openai
import matplotlib.pyplot as plt
import numpy as np
openai.api_key = "sk-LopWyMKqQ2aCySPMeFSZLyrAur0a8EACzgbHBXMK"
from sklearn.datasets import fetch_openml

from sklearn import tree



results=[]

# In[108]:


from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
mdatos=df
#remove missing
mdatos=mdatos.dropna()
#remove classes with few elements


In [17]:


#cl="race"
for cl in  mdatos.columns[13:]:
  
  print("column:" +cl)
  
  classif=False
  if (mdatos[cl].dtype=="O"):
      classif=True
      datos=mdatos.groupby(cl).filter(lambda x: len(x) > 10)
  else:
      classif=False
      datos=mdatos

  datos = datos.sample(frac=1).reset_index(drop=True)
  X=datos.loc[:, datos.columns != cl] # Features
  y=datos[cl]
   
  
  
  if (classif): 
      X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
  else:
          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
      
  
  
  # In[109]:
  
  
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.impute import SimpleImputer
  from sklearn.compose import ColumnTransformer
  cols = datos.columns
  
  num_cols = X._get_numeric_data().columns
  categorical_columns  = [col for col in X.columns if X[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  #categorical_columns = X._get_numeric_data().columns
  
  
  #X = X[categorical_columns + numerical_columns]
  
  #X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
  
  
  categorical_encoder = OneHotEncoder(handle_unknown="ignore")
  numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean"))])
  
  preprocessing = ColumnTransformer(
      [
          ("cat", categorical_encoder, categorical_columns),
          ("num", numerical_pipe, numerical_columns),
      ]
   )
  
  if (classif):
   rf = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeClassifier(random_state=42)),
      ]
   )
  else:
   rf = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeRegressor(random_state=42)),
      ]
   ) 
  
  
  rf.fit(X_train, y_train)
  
  
  # In[110]:
  
  
  
 # if (classif):
#    print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
#    print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))
#  else:
#    print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
#    print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))
  
  
  # In[111]:
  
  
  from sklearn.inspection import permutation_importance
  result = permutation_importance(
      rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
  )
  sorted_idx = result.importances_mean.argsort()
  
  #fig, ax = plt.subplots()
  #ax.boxplot(
  #    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
  #)
  #ax.set_title("Permutation Importances (test set)")
  #fig.tight_layout()
  #plt.show()
  
  
  # In[112]:
  
  
  nsel=1
  selected=list(X_test.columns[sorted_idx][-nsel:])
  
  
  # In[113]:
  
  
  Xn=datos.loc[:, selected] # Features 
  
  
  # In[114]:
  
  
  
  num_cols = Xn._get_numeric_data().columns
  categorical_columns  = [col for col in Xn.columns if Xn[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  categorical_encoder = OneHotEncoder(handle_unknown="ignore")
  numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean"))])
  
  preprocessing = ColumnTransformer(
      [
          ("cat", categorical_encoder, categorical_columns),
          ("num", numerical_pipe, numerical_columns),
      ]
   )
  
  if (classif):
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeClassifier(random_state=42)),
      ]
   )
  else:
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeRegressor(random_state=42)),
      ]
   ) 
  
  
  
  
  # In[115]:
  
  
  n=5 #repetitions
  tam=200 #prompt
  
  responses=[]
  correct=[]
  responsesg=[]
  
  for ij in range(n):
    #  print (ij)
     
      if (classif):
       X1_train, X1_test, y1_train, y1_test = train_test_split(Xn, y,train_size=tam, stratify=y, random_state=ij)
      else:
       X1_train, X1_test, y1_train, y1_test = train_test_split(Xn, y,train_size=tam, random_state=ij)
      ### DT
      dt.fit(X1_train, y1_train)
      predictions = dt.predict(X1_test)
      predictions[0]==y1_test.iloc[0]
      #print(predictions[0])
      #print(y1_test.iloc[0])
      correct.append(y1_test.iloc[0]) 
      responses.append(predictions[0])
      ### GPT3
      att=X1_train.columns[0]
      prompt=""
      #Build Prompt
      for i in range (0,(tam-1)):
              prompt=prompt+"input:"+str(X1_train[att].iloc[i])+', output:'+str(y1_train.iloc[i])+'\n'
      prompt1=prompt+" input:"+str(X1_test[att].iloc[0])+', output:'
      response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt1,
        temperature=0,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
          )
      #Response
     # if (response["choices"][0]["text"].split('\n')[0]==y1_test.iloc[0]):
     #     accg=accg+1
      responsesg.append(response["choices"][0]["text"].split('\n')[0])   
       
  
  
  # In[116]:
  
  
  acc=0
  accg=0
  if classif:
   for i in range(n):  
          if (responses[i]==correct[i]):
              acc=acc+1
          if (responsesg[i]==correct[i]):
              accg=accg+1
  
  if not(classif):
   for i in range(n):
          acc=acc+abs(responses[i]-correct[i])
          accg=accg+abs(float(responsesg[i])-correct[i])
               
  
  
  # In[117]:
  
  if classif:  
   print("accuracy gpt3")
   print(float(accg)/n)
   print("accuracy DT")
   print(float(acc)/n)
   results.append((cl,n,float(accg)/n,float(acc)/n))

  if not(classif):
    print("MAE gpt3")
    print(float(accg)/n)
    print("MAE DT")
    print(float(acc)/n)
    results.append((cl,n,float(accg)/n,float(acc)/n))

  
  
  
  


column:Month Number
MAE gpt3
0.0
MAE DT
0.0
column:Month Name
accuracy gpt3
1.0
accuracy DT
1.0
column:Year
MAE gpt3
0.2
MAE DT
0.38552631578945695


In [11]:
 mdatos.columns[13:]

Index(['Month Number', 'Month Name', 'Year'], dtype='object')

In [18]:
results


[('Segment', 5, 1.0, 1.0),
 ('Country', 5, 0.4, 0.2),
 ('Product', 5, 1.0, 1.0),
 ('Discount Band', 5, 0.6, 0.8),
 ('Units Sold', 5, 1019.1, 976.3843722943724),
 ('Manufacturing Price', 5, 0.0, 0.0),
 ('Sale Price', 5, 46.6, 46.0),
 ('Gross Sales', 5, 15367.9, 11476.8),
 ('Discounts', 5, 22396.129999999997, 21407.058),
 (' Sales', 5, 14752.207999999999, 5576.33),
 ('COGS', 5, 6064.0, 11898.6),
 ('Profit', 5, 19209.0, 12964.380000000001),
 ('Month Number', 5, 0.0, 0.0),
 ('Month Name', 5, 1.0, 1.0),
 ('Year', 5, 0.2, 0.38552631578945695)]

In [9]:
responses

[210489.04109589042]

In [14]:
mdatos.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       ' Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')